<a href="https://www.kaggle.com/code/masatomurakawamm/recommendation-systems-overview?scriptVersionId=150761839" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


---
### 【SVD, MF (Matrix Factorization), IMF (Implicit MF), and iALS (implicit Alternating Least Square) 】  

Assuming that you have a feedback matrix $ X \in \mathbb R^{m \times n}$, where $m$ is the number of users (or queries) and $n$ is the number of items.
These algorithms learn:
  - A user embedding matrix $U \in \mathbb R^{m \times d}$, where row $i$ is the embedding for user $i$.  
  - An item embedding matrix $I \in \mathbb R^{n \times d}$, where row $j$ is the embedding for item $j$.  
$d$ is the hyperparameter which represents the number of latent factors.

 <img width="1000" src="https://developers.google.com/machine-learning/recommendation/images/Matrixfactor.svg">  
 
(https://developers.google.com/machine-learning/recommendation/collaborative/matrix)

Majoir differences between these four algorithms are objective functions to minimize. In addition, whether the feedback is explicit or implicit shold be considered to choose solution.　Explicit feedback is a form of feedback where users directly express their opinions or evaluations, numerical ratings　or assessments for example. Implicit feedback is indirect feedback obtained from user actions and interactions. For example, clicking on specific products, content or search results, and so on.


- **SVD**  
feedback: explicit  
data preprocessing: NaN is replaced to '0' or 'mean value'.  
objective function:  

$$
\begin{eqnarray}
L(U, V) =  \sum_{i, j} (X_{ij} - \langle U_{i}, I_{j} \rangle)^2 \\
\end{eqnarray}
$$  

  note: SVD is effective when the feedback matrix is dense. However, when the feedback matrix is sparse, the prediction values with SVD would be close to '0' (or 'mean value'). 


- **Matrix Factorization**  
feedback: explicit  
data preprocessing: NaN is not replaced.  
objective function: 

$$
\begin{eqnarray}
L(U,V) &=& \sum_{(i, j) \in \text{obs}} (X_{ij} - \langle U_{i}, I_{j} \rangle)^2 +\lambda\left(\sum_i \|U_i\|^2 +\sum_j\|I_j\|^2\right) \\ 
\end{eqnarray}
$$

  The second term is for L2 Regularization, and $\lambda$ is a hyperparameter which controls the importance of regularization.

  note: The point of Matrix Factorization is to ignore NaN values. This improves the weakness of SVD.


- **IMF**  
feedback: implicit   
data preprocessing: NaN is not replaced.  
objective function: 

$$
\begin{eqnarray}
L(U,V) 
&=& \sum_{i, j} c_{ij} (\bar{r_{ij}} - \langle U_{i}, I_{j} \rangle)^2 + \lambda \left( \sum_{i} ||U_i||^2 + \sum_{j} ||I_j||^2 \right) \\
\end{eqnarray}
$$  

  IMF considers the number of views, or $r_{ij}$; the number of views of user $i$ for item $j$. The binary variable, $\bar{r_{ij}}$ is calculated based on $r_{ij}$ as follows, and represents the preference of user $i$ to item $j$.  

$$
\begin{eqnarray}
\bar{r_{ij}} = 
    \begin{cases}
        {1 \ (r_{ij} > 0)} \\
        {0 \ (r_{ij} = 0)} \\
    \end{cases}
\end{eqnarray}
$$  

  Additionally, the confidence of the preference, or $c_{ij}$, is also calculated based on $r_{ij}$. Confidence increases proportionally to $r_{ij}$, and $\alpha$ is a hyperparameter.  

$$
\begin{eqnarray}
c_{ij} = 1 + \alpha r_{ij} \\
\end{eqnarray}
$$  

  When the feedback is implicit, non-observed data points are treated as negative examples, and all the data points (user $\times$ items) are summed for the evaluation. 


- **iALS**   
feedback: implicit  
data preprocessing: NaN is not replaced.  
objective function: 

$$
\begin{eqnarray}
L(U,V) &=& \sum_{i, j} (X_{ij} - \langle U_{i}, I_{j} \rangle)^2 +\lambda\left(\sum_i \|U_i\|^2 +\sum_j\|I_j\|^2\right) \\
&=& \sum_{(i, j) \in \text{obs}} (\langle U_{i}, I_{j} \rangle - 1)^2 + w_0 \sum_{(i, j) \not \in \text{obs}} (\langle U_i, I_j\rangle - 0)^2 +  \lambda\left(\sum_i \|U_i\|^2 +\sum_j\|I_j\|^2\right) \\
&=& \sum_{(i, j) \in \text{obs}} (\langle U_{i}, I_{j} \rangle - 1)^2 + w_0 \sum_{(i, j) \not \in \text{obs}} \langle U_i, I_j\rangle^2 +  \lambda\left(\sum_i \|U_i\|^2 +\sum_j\|I_j\|^2\right) \\
\end{eqnarray}
$$ 

  note: The second term is a sum over unobserved entries (treated as zeroes), and $w_0$ is a hyperparameter that controls the importance of observed and unobserved entries.


---
  By using these objective functions, $U$ and $V$ are computed with SGD(Stochastic Gradient Descent) or ALS(Alternating Least Squares).

---

### 【Factorization Machines (FM)】

The data structure of Factorization Machines is different from that of SVD. IDs of users and items are treated as categorical (one-hot) columns. Features of attribute information(for example, age of users or price of items) and evaluation values (or labels) are also added as column features and answer label, as respectively. As a result, one evaluation sample is treated as one data point, as follows:

| sample | user 1 | user 2 | … | item 1 | item 2 | … | feature 1 | feature 2 | … | label |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| $x^{1}$ | 1 | 0 | … | 1 | 0 | … | 0.6 | 1.1 | … | $y^{1}$ |
| $x^{2}$ | 1 | 0 | … | 0 | 1 | … | 0.2 | 0.8 | … | $y^{2}$ |
| $x^{3}$ | 1 | 0 | … | 0 | 0 | … | 0.7 | 0.1 | … | $y^{3}$ |
| $x^{4}$ | 0 | 1 | … | 1 | 0 | … | 0.4 | 0.2 | … | $y^{4}$ |
| $x^{5}$ | 0 | 1 | … | 0 | 1 | … | 0.3 | 0.5 | … | $y^{5}$ |
| … | … | … | … | … | … | … | … | … | … | … |
| $x^{n}$ | 0 | 0 | … | 0 | 0 | … | 1.0 | 0.9 | … | $y^{n}$ |

- The most simple FM predicts $\hat{y}$ as following linear model:

$$
\begin{aligned} 
\hat{y^{(i)}} = \left\langle \boldsymbol{w},\boldsymbol{x^{(i)}}\right\rangle = w_0 + \sum_{j=1}^{D}w_{j}x^{(i)}_{j}\end{aligned}
$$

$$
\boldsymbol{w}\in\mathbb{R}^{D},
\boldsymbol{x}\in\mathbb{R}^{n \times D}
$$

$\boldsymbol{w}$ is a parameter vector to learn.

- Assuming that you add the interaction term to the linear model. The simple solution is as follows:

$$
\begin{aligned} 
\hat{y^{(i)}} = \left\langle \boldsymbol{w},\boldsymbol{x^{(i)}}\right\rangle +\sum_{l,m}w_{l,m}x^{(i)}_{l}x^{(i)}_{m}\end{aligned} 
$$

However, this model contains $D \times D$ parameters, and sparsity of feedback makes it hard to learn the whole interaction parameters.

- More common (quadratic) FM predicts $\hat{y}$ as follows:

$$
\begin{aligned} 
\hat{y^{(i)}} = & w_0 + \left\langle \boldsymbol{w},\boldsymbol{x^{(i)}}\right\rangle +\sum_{l=1}^{D}\sum_{m=l+1}^{D}\left\langle \boldsymbol{v}_{l},\boldsymbol{v}_{m}\right\rangle x^{(i)}_{l}x^{(i)}_{m}\nonumber \\ 
= & w_0 + \left\langle \boldsymbol{w},\boldsymbol{x^{(i)}}\right\rangle +\sum_{(l, m)\in C}\left\langle \boldsymbol{v}_{l},\boldsymbol{v}_{m}\right\rangle x^{(i)}_{l}x^{(i)}_{m}\end{aligned}
$$

$v_{l}, v_{m} \in\mathbb{R}^{k}$ are called latent vectors. $k$ is a hyperparameter of embedding dimention. FM represents the importance of interactions between $x_{l}$ and $x_{m}$ as the inner product of $v_{l}$ and $v_{m}$, $\left\langle v_{l}, v_{m}\right\rangle$. This helps to learn indirectly the weights of interactions which is not observed. Moreover, computation of interaction parts takes $O(kD)$, not $O(kD^2)$, because of formula transformation.

- **Field-Aware Factorization Machines (FFM)**

In FM, interactions of features are equally represented as inner products of latent vectors, $v$. FFM added a new index to $v$, and represents the prediction of $y$ as follows:

$$
\hat{y^{(i)}} = w_0 + \sum_j w_j x_j^{(i)} + \sum_{(l, m)\in C} \langle v_{l,F(m)}, v_{m,F(l)} \rangle x_l x_m
$$

$F_{l}$ is the field which $x_{l}$ belongs to, and $v_{l,F(m)}$ is the embedding of $x_{l}$ for the field $F_{m}$. FFM has the same number of latend embeddings as the number of fields, and learns all of them. This achieves the representations of interactions with factors unique to each field. However, you must note that there is a risk of overfitting (and there are many countermeasures, of course).

FM and FFM are widely applied with many kinds of improvements, such as combination with DNN.